# 📘 Day 33: Accessing Web APIs with `requests`

## Overview

This lesson introduces a lightweight wrapper around the
[JSONPlaceholder](https://jsonplaceholder.typicode.com/) demo API. The module
exports three helpers—`fetch_users`, `fetch_post`, and `fetch_posts_by_user`—that
return structured data for analytics workflows while keeping the HTTP layer easy
to mock during tests.

## Learning goals

- Understand how to issue HTTP GET requests with `requests`.
- Convert JSON payloads into `pandas.DataFrame` objects for analysis.
- Inject custom HTTP clients (for example, `requests.Session` objects or call
  stubs) to make networked code testable.

## Requirements

Install the core and testing dependencies into your environment:

```bash
pip install -r requirements.txt
pip install pytest responses
```

`responses` is only needed when you want to simulate the API locally or run the
pytest suite.

## Running the lesson

### Live API demo

Execute the script directly to fetch data from JSONPlaceholder:

```bash
python Day_33_API/api.py
```

The command prints a preview of the user list, details for post `1`, and a table
of posts authored by user `2`.

### Mocked endpoints

The helper functions accept a `requests.Session` or any callable with the same
signature as `requests.get`. This allows you to supply canned responses when the
internet is unavailable or you want deterministic examples:

```python
from Day_33_API.api import fetch_users

class MockClient:
    def get(self, url, **kwargs):
        class _Response:
            def __init__(self, payload):
                self._payload = payload

            def raise_for_status(self):
                return None

            def json(self):
                return self._payload

        return _Response([{"id": 1, "name": "Ada", "username": "ada"}])

mocked_users = fetch_users(client=MockClient())
print(mocked_users)
```

The included pytest suite (see below) relies on the `responses` library to
provide richer, request-aware mocks if you prefer a declarative API.

## Tests

Run the Day 33 unit tests, which exercise both success and error paths using
mocked HTTP responses:

```bash
pytest tests/test_day_33.py
```

To execute the entire collection of lesson tests, run `pytest` from the project
root.

Day 33: Solutions to Exercises

In [ ]:

import pandas as pd
import requests

# --- Exercise 1: Fetch All Posts ---
print("--- Solution to Exercise 1 ---")
posts_url = "https://jsonplaceholder.typicode.com/posts"
try:
    response = requests.get(posts_url)
    response.raise_for_status()
    posts_data = response.json()
    posts_df = pd.DataFrame(posts_data)
    print("DataFrame of all posts (first 5 rows):")
    print(posts_df.head())
except requests.exceptions.RequestException as e:
    print(f"Error fetching posts: {e}")
print("-" * 20)


# --- Exercise 2: Fetch a Specific User's Data ---
print("--- Solution to Exercise 2 ---")
user_id = 5
user_url = f"https://jsonplaceholder.typicode.com/users/{user_id}"
try:
    response = requests.get(user_url)
    response.raise_for_status()
    user_data = response.json()
    # The 'company' value is a nested dictionary
    company_name = user_data.get("company", {}).get("name", "N/A")
    print(f"Data for User ID {user_id}:")
    print(f"  Name: {user_data.get('name', 'N/A')}")
    print(f"  Company: {company_name}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching user {user_id}: {e}")
print("-" * 20)


# --- Exercise 3: Fetch Comments for a Specific Post ---
print("--- Solution to Exercise 3 ---")
comments_url = "https://jsonplaceholder.typicode.com/comments"
params = {"postId": 3}
try:
    response = requests.get(comments_url, params=params)
    response.raise_for_status()
    comments_data = response.json()
    comments_df = pd.DataFrame(comments_data)
    print(f"DataFrame of comments for postId={params['postId']} (first 5 rows):")
    print(comments_df.head())
except requests.exceptions.RequestException as e:
    print(f"Error fetching comments: {e}")
print("-" * 20)
